# Geomatching

This notebook shows how to use the `Geomatcher` object in `constructive_geometries`.

## Consistent global topology

The foundation for the `geomatcher` is a consistent global topology, built using the [Constructive Geometries](https://bitbucket.org/cmutel/constructive-geometries) repository, and provided as a Python library here. The base data is from [Natural Earth](http://www.naturalearthdata.com/), though there has been a number of fixes and added locations.

A [topolgy](https://postgis.net/docs/Topology.html) is useful because it comes with guarantees of consistency, as each edge is only stored once. This allows us to do GIS operations using set algebra with the ids of topological faces. Here is an part of the world, as provided in `Constructive Geometries`:

<img src="japan.png">

As you can see, each polygon is a face, no matter how big or small it is.

## Retrieving facial ids

As a first try, you can retrieve the faces associated with any given location:

In [1]:
from constructive_geometries import *
geomatcher = Geomatcher()
list(geomatcher['JP'])[:10]

[6144, 6145, 6146, 6147, 6148, 6149, 6150, 6151, 6152, 6153]

Geospatial definitions are namespaced, except for countries. Countries are therefore defined by their ISO two-letter codes, but other data should be referenced by a tuple of its namespace and identifier:

In [2]:
list(geomatcher[('ecoinvent', 'NAFTA')])[:10]

[2048, 1, 2, 3, 4, 5, 6, 7, 8, 9]

You can also set a default namespace. The default is `"ecoinvent"`. So, the IMAGE regions are loaded by default, and must be retrieved explicitly, while ecoinvent regions will be searched automatically, unless the default namespace is changed.

In [3]:
list(geomatcher['NAFTA'])[:10]

[2048, 1, 2, 3, 4, 5, 6, 7, 8, 9]

Finally, by default `geomatcher` will search for country names, using [country converter](https://github.com/konstantinstadler/country_converter).

In [4]:
'Japan' in geomatcher.topology

False

In [5]:
list(geomatcher['Japan'])[:10]

Geomatcher: Used 'JP' for 'Japan'


[6144, 6145, 6146, 6147, 6148, 6149, 6150, 6151, 6152, 6153]

## GIS operations: intersection, contained, within

Geomatcher allows you to do quick GIS calculations.

In [6]:
geomatcher.intersects("US")

['GLO',
 ('ecoinvent', 'UN-AMERICAS'),
 ('ecoinvent', 'RNA'),
 ('ecoinvent', 'NAFTA'),
 ('ecoinvent', 'IAI Area 2, North America'),
 ('ecoinvent', 'IAI Area 2, without Quebec'),
 ('ecoinvent', 'US-ASCC'),
 ('ecoinvent', 'NPCC'),
 ('ecoinvent', 'US-NPCC'),
 ('ecoinvent', 'US-HICC'),
 ('ecoinvent', 'WECC'),
 ('ecoinvent', 'US-WECC'),
 ('ecoinvent', 'US-SERC'),
 ('ecoinvent', 'US-RFC'),
 ('ecoinvent', 'MRO'),
 ('ecoinvent', 'US-TRE'),
 ('ecoinvent', 'US-FRCC'),
 ('ecoinvent', 'US-MRO'),
 ('ecoinvent', 'US-SPP')]

`contained` gets all locations that are *completely within* this location, whereas `within` gets all locations that *completely contain* this location.

In [8]:
geomatcher.contained("US")

['US',
 ('ecoinvent', 'US-ASCC'),
 ('ecoinvent', 'US-NPCC'),
 ('ecoinvent', 'US-HICC'),
 ('ecoinvent', 'US-WECC'),
 ('ecoinvent', 'US-SERC'),
 ('ecoinvent', 'US-RFC'),
 ('ecoinvent', 'US-FRCC'),
 ('ecoinvent', 'US-MRO'),
 ('ecoinvent', 'US-SPP')]

In [9]:
geomatcher.within("US")

['GLO',
 ('ecoinvent', 'UN-AMERICAS'),
 ('ecoinvent', 'RNA'),
 ('ecoinvent', 'NAFTA'),
 ('ecoinvent', 'IAI Area 2, North America'),
 ('ecoinvent', 'IAI Area 2, without Quebec'),
 'US']

For all three operations, you can exclude the input variable:

In [10]:
geomatcher.within("US", include_self=False)

['GLO',
 ('ecoinvent', 'UN-AMERICAS'),
 ('ecoinvent', 'RNA'),
 ('ecoinvent', 'NAFTA'),
 ('ecoinvent', 'IAI Area 2, North America'),
 ('ecoinvent', 'IAI Area 2, without Quebec')]

You can also change the sorting order, with is biggest first by default.

In [11]:
geomatcher.within("US", include_self=False, biggest_first=False)

[('ecoinvent', 'IAI Area 2, without Quebec'),
 ('ecoinvent', 'IAI Area 2, North America'),
 ('ecoinvent', 'NAFTA'),
 ('ecoinvent', 'RNA'),
 ('ecoinvent', 'UN-AMERICAS'),
 'GLO']

Finally, you can ask for an list where none of the regions overlap:

In [12]:
geomatcher.intersects("US", biggest_first=False, exclusive=True)

[('ecoinvent', 'US-FRCC'),
 ('ecoinvent', 'US-MRO'),
 ('ecoinvent', 'US-SPP'),
 ('ecoinvent', 'US-TRE'),
 ('ecoinvent', 'US-RFC'),
 ('ecoinvent', 'US-SERC'),
 ('ecoinvent', 'US-WECC'),
 ('ecoinvent', 'US-HICC'),
 ('ecoinvent', 'US-NPCC'),
 ('ecoinvent', 'US-ASCC')]

### Rest of World and `resolved_row`

By default, the location 'Rest of World' or 'RoW' does not have a fixed spatial definition - it is defined dynamically by whatever is left over some regions are defined. So `RoW("France")` would be the rest of the world, aside from France. As such, 'RoW' is never returned by the GIS functions in `geomatcher`, unless it is defined.

You can use the `resolved_row` context manager to define the rest of the world temporarily:

In [9]:
assert 'RoW' not in geomatcher

with resolved_row(["FR", "GB"], geomatcher) as g:
    assert 'RoW' in g
    
assert 'RoW' not in geomatcher

In this case, the 'US' is not in the 'RoW':

In [11]:
with resolved_row(["NAFTA"], geomatcher) as g:
    print(geomatcher.within("US"))

['GLO', ('ecoinvent', 'UN-AMERICAS'), ('ecoinvent', 'RNA'), ('ecoinvent', 'NAFTA'), ('ecoinvent', 'IAI Area 2, North America'), ('ecoinvent', 'IAI Area 2, without Quebec'), 'US']


When the 'RoW' is a valid response, it is treated the same as any other location - it has a set of faces, so will be sorted in the correct order.

In [12]:
with resolved_row(["FR"], geomatcher) as g:
    print(geomatcher.within("US"))

['GLO', 'RoW', ('ecoinvent', 'UN-AMERICAS'), ('ecoinvent', 'RNA'), ('ecoinvent', 'NAFTA'), ('ecoinvent', 'IAI Area 2, North America'), ('ecoinvent', 'IAI Area 2, without Quebec'), 'US']


In [14]:
with resolved_row(["FR"], geomatcher) as g:
    print(geomatcher.within("US", biggest_first=False))

['US', ('ecoinvent', 'IAI Area 2, without Quebec'), ('ecoinvent', 'IAI Area 2, North America'), ('ecoinvent', 'NAFTA'), ('ecoinvent', 'RNA'), ('ecoinvent', 'UN-AMERICAS'), 'RoW', 'GLO']


## Splitting faces

Say we wanted to split the island of Honshu and develop a separate inventory for Tokyo. From our graphic above, we know that Honshu has face number 6247. So, we can split this face into two new ones - one of which we will consider Tolyo, and the other the "Rest of Honshu".

In [13]:
first, second = geomatcher.split_face(6247)
first, second

(7162, 7163)

In [14]:
geomatcher.topology[("example", "Rest of Honshu")] = set([first])
geomatcher.topology[("example", "Tokyo")] = set([second])

In [15]:
geomatcher.contained("JP")

['JP', ('example', 'Rest of Honshu'), ('example', 'Tokyo')]

`split_face` also supports the arguments `number` (number of new faces to create), and `ids` (integers values for the new ids to create). If `ids` is passed, `number` is ignored.

## Adding new topologies

You can also add new topologies to support custom spatial systems. This is how the IMAGE regions are added in Wurst:

    geomatcher.add_definitions(IMAGE_TOPOLOGY, "IMAGE")
    
New topologies can be either relative (default) or not. Relative topologies are defined by reference to regions already in the topology:

    {"Russia Region": [
        "AM",
        "AZ",
        "GE",
        "RU"
    ]}
    
Non-relative topologies must be defined by a set of integer ids.

    {
        'A': {1, 2, 3},
        'B': {2, 3, 4},
    }
    
Regions added by `add_definitions` will be namespaced with the second argument passed to the function.

In [16]:
topoz = {"Black Sea": [
    "RO",
    "TR",
    "UA",
    "GE",
    "BG",
    "RU",
]}
geomatcher.add_definitions(topoz, "just added")

In [17]:
geomatcher.contained(("just added", "Black Sea"))

[('just added', 'Black Sea'),
 'RU',
 ('ecoinvent', 'Russia (Asia)'),
 ('ecoinvent', 'Russia (Europe)'),
 'TR',
 'UA',
 'BG',
 'GE',
 'RO']